In [ ]:
from load import load_sql

In [ ]:
msg=load_sql()

In [ ]:
msg.head(100)

In [ ]:
# Cleaning
def no_forward(df):
    return df[~df.body.str.startswith("---------------------- Forwarded by")]


def no_original_msg(df):
    df = df.copy()
    exprs = [
        "-----Original Message",
        "---------------------- Forwarded by",
        "----- Forwarded by",
        "-------- Inline attachment follows",
        "\t\t\tFrom:",
        "cc: Subject:",
        "Sara ShackletonEnron North America Corp",
        " Debra PerlingiereEnron",
        "Outlook Migration Team",
        "Matthew B Fleming@",
        "Lynn Lynn Blair Northern",
        "TASK ASSIGNMENT",
        "X-FileName",
        "KayRhonda L Denton@ECT0",
    ]
    # Find emails to drop
    drop_emails = (
        df.body.str.extract('("?[A-z \.,]+"? <[a-zA-Z0-9_.]+@[A-z ]+\..{2,3}>)')[0]
        .dropna()
        .unique()
        .tolist()
    )
    exprs += drop_emails
    drop_emails2 = (
        df.body.str.extract("((?:From:)? [A-z ]+@[A-Z]{1,7})").dropna()[0].unique().tolist()
    )
    exprs += drop_emails2
    drop_emails3 = (
        df.body.str.extract("([0-9]{2}/[0-9]{2}/[0-9]{4} [0-9]{2}:[0-9]{2} PMTo:)")
        .dropna()[0]
        .unique()
        .tolist()
    )
    exprs+=drop_emails3
    for expr in exprs:
        mask = df.body.str.contains(expr,regex=False)

        df.loc[mask, "body"] = df.loc[mask, "body"].apply(lambda x: x.split(expr)[0])
    return df


def remove_words(df):
    df = df.copy()
    reg = {
        "--------------------------Sent from my BlackBerry Wireless Handheld \(www\.BlackBerry\.net\)": "",
        "\t": " ",
        " +": " ",
    }
    for exp, replace in reg.items():
        df["body"] = df.body.str.replace(exp, replace, regex=True)
    return df


def remove_blank(df):
    df["body"] = df["body"].str.strip()
    return df[df.body.str.len() >= 2]


def process_df(df):
    return (
        df.pipe(no_forward).pipe(no_original_msg).pipe(remove_words).pipe(remove_blank)
    )


t = msg.pipe(process_df)

In [ ]:
!mkdir processed_data

In [ ]:
t.to_csv('processed_data/data.csv',index=False)

In [ ]:
t.body.sample(5).apply(lambda x: print(x,"\n"))

In [ ]:
t2=t[t.sender.isin(t.groupby('sender').count().pipe(lambda x: x[x.body>=50]).index)]


In [ ]:
t2.assign(l=lambda x: x.body.str.len()).groupby("sender").l.agg(
    ["mean", "count"]
).sort_values("mean")

In [ ]:
t2[t2.sender=="mike.maggi@enron.com"].body.sample(5).apply(print)

In [ ]:
# Drink-> party words?
t2[t2.body.str.contains("drink")]

In [ ]:
import re
t2[(t2.body.str.len()<100)&(t2.body.str.contains("can you",flags=re.I))]

In [ ]:
t2[t2.body.str.contains("horse")].sample(1).body.iloc[0]